In [13]:
from IPython.display import Image, display, HTML, JSON
import pandas as pd
import ast
from dotenv import load_dotenv
import re

#### Load Learning Examples & Testing Products

In [14]:
def convert_string_to_list(string_list):
    try:
        actual_list = ast.literal_eval(string_list)
        return actual_list
    except ValueError as e:
        print(f"Error converting string to list: {e}")
        return None

In [15]:
examples_file_path = 'examples.csv'
examples_df = pd.read_csv(examples_file_path)
examples_df['product_images'] = examples_df['product_images'].apply(convert_string_to_list)
examples_df.head()

,address,product_title,description,product_images
0,https://www.pazzion.com/products/191-5a-fiorel...,Fiorella Boots,"A perfect blend of style, comfort, and conveni...",[https://www.pazzion.com/cdn/shop/files/Fiorel...
1,https://www.pazzion.com/products/2203a-1-mered...,Meredith Ankle Lace-up Boots,The perfect blend of style and comfort for the...,[https://www.pazzion.com/cdn/shop/files/Meredi...
2,https://www.pazzion.com/products/2597-1-ingrid...,Ingrid Gold Studded Cage Patent Leather Slides,Gladiator styled sandals made in glossy patent...,[https://www.pazzion.com/cdn/shop/files/Ingrid...
3,https://www.pazzion.com/products/522-1-kate-un...,Kate Unlace Platform Sneakers,Athletic trainers don’t need to be dedicated t...,[https://www.pazzion.com/cdn/shop/files/KateUn...
4,https://www.pazzion.com/products/80-29-reagan-...,Reagan Contrast Sneakers,A pristine clean white sneaker is essential to...,[https://www.pazzion.com/cdn/shop/files/Reagan...


In [16]:
for column, value in examples_df.iloc[0].items():
    print(f"{column}: {value}")

address: https://www.pazzion.com/products/191-5a-fiorella-boots
product_title: Fiorella Boots
description: A perfect blend of style, comfort, and convenience, crafted to cater to the modern woman's needs, Fiorella is designed with a focus on ease of wear and contemporary style. These boots feature a convenient zipper closure, making them a breeze to put on and take off. No more struggling with laces or buttons; whether you're dashing through your daily routine or stepping out for a night on the town, just zip up and you're ready to conquer your day.
product_images: ['https://www.pazzion.com/cdn/shop/files/FiorellaBoots-Camel-1.webp?v=1705915480', 'https://www.pazzion.com/cdn/shop/files/FiorellaBoots-Camel-10.webp?v=1715759531', 'https://www.pazzion.com/cdn/shop/files/FiorellaBoots-Camel-2.webp?v=1715759531', 'https://www.pazzion.com/cdn/shop/files/FiorellaBoots-Camel-3.webp?v=1715759531', 'https://www.pazzion.com/cdn/shop/files/FiorellaBoots-Camel-4.webp?v=1715759531']


In [17]:
products = 'test_products.csv'
products_df = pd.read_csv(products)
products_df['product_images'] = products_df['product_images'].apply(convert_string_to_list)
products_df.head()

,address,product_title,product_images
0,https://www.pazzion.com/products/c2257-3-joyce...,Joyce Pearl Patent Slingback Heels,[https://www.pazzion.com/cdn/shop/files/JoyceP...
1,https://www.pazzion.com/products/122-5-maia-pe...,Maia Pearl Décor Bow Slingbacks Kitten Heels,[https://www.pazzion.com/cdn/shop/files/MaiaPe...
2,https://www.pazzion.com/products/6313-1-aureli...,Aurelia Strappy Espadrilles,[https://www.pazzion.com/cdn/shop/files/Aureli...
3,https://www.pazzion.com/products/1919-1-margot...,Margot Tweed Lace-up Espadrilles,[https://www.pazzion.com/cdn/shop/files/Margot...
4,https://www.pazzion.com/products/3899-9-fallon...,Fallon Leather Ankle Boots,[https://www.pazzion.com/cdn/shop/files/Fallon...


#### Create a Few-Shot Prompt

In [18]:
with open('prompts/copywriting_system_prompt_v2.txt', 'r') as file:
    copywriting_system_prompt_v2 = file.read()

In [19]:
def create_example_prompt(df):
    example_prompt = []

    for index, row in df.iterrows():
        product_title = row['product_title']
        product_description = row['description']
        product_images = row['product_images']


        user_content = [{"type": "text", "text": f"<product_title>{product_title}</product_title>"}]

        for image_url in product_images:
            user_content.append({
                "type": "image_url",
                "image_url": {
                    "url": image_url,
                }
            })

        example_prompt.append({
            "role": "user",
            "content": user_content
        })
        
        example_prompt.append({
            "role": "assistant",
            "content": [
                {"type": "text", "text": "<product_description>"+product_description+"</product_description>"}
            ]
        })
    
    return example_prompt

In [20]:
def create_user_prompt(row):
    user_prompt = []

    product_title = row['product_title']
    product_images = row['product_images']
    
    user_content = [{"type": "text", "text": f"<product_title>{product_title}</product_title>"}]

    for image_url in product_images:
        user_content.append({
            "type": "image_url",
            "image_url": {
                "url": image_url,
            }
        })
    
    user_prompt.append({
            "role": "user",
            "content": user_content
        })
    
    return user_prompt

In [21]:
## Visualize the prompt (message_list)
## https://jsoneditoronline.org/ 

first_product = products_df.iloc[0]

example_prompt = create_example_prompt(examples_df)
user_prompt = create_user_prompt(first_product)

copywriter_message_list = [{"role": "system", "content": copywriting_system_prompt_v2}] + example_prompt + user_prompt
copywriter_message_list


[{'role': 'system',
  'content': 'You are an AI assistant tasked with generating product descriptions based on the provided product title and images.\n\nPresent your product description within XML tags as follows:\n<product_description>\n[Your written product description goes here]\n</product_description>'},
 {'role': 'user',
  'content': [{'type': 'text',
    'text': '<product_title>Fiorella Boots</product_title>'},
   {'type': 'image_url',
    'image_url': {'url': 'https://www.pazzion.com/cdn/shop/files/FiorellaBoots-Camel-1.webp?v=1705915480'}},
   {'type': 'image_url',
    'image_url': {'url': 'https://www.pazzion.com/cdn/shop/files/FiorellaBoots-Camel-10.webp?v=1715759531'}},
   {'type': 'image_url',
    'image_url': {'url': 'https://www.pazzion.com/cdn/shop/files/FiorellaBoots-Camel-2.webp?v=1715759531'}},
   {'type': 'image_url',
    'image_url': {'url': 'https://www.pazzion.com/cdn/shop/files/FiorellaBoots-Camel-3.webp?v=1715759531'}},
   {'type': 'image_url',
    'image_url': 

## LLM

#### GPT-4o

In [5]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
client = OpenAI()

##### Copywriter

In [6]:
def describe_image(message_list):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=message_list,
            max_tokens=4000,
            temperature=0.8 # 0~1, higher value, more creative 
        )
        return response.choices[0].message.content
    except Exception as e:
        return str(e)
    # return response.choices[0].message.content, response.usage

In [7]:
def extract_product_description(xml_string):
    match = re.search(r'<product_description>\s*(.*?)\s*</product_description>|<revised_description>\s*(.*?)\s*</revised_description>', xml_string, re.DOTALL)
    if match:
        return match.group(1).strip() if match.group(1) else match.group(2).strip()
    return None

In [ ]:
first_product = products_df.iloc[0]

print(f"Product Title: "+ first_product['product_title'])
print(f"Address: "+ first_product['address'])
print("Product Images:")
for image_url in first_product['product_images']:
    print(image_url)
    
display(Image(url=first_product['product_images'][0], width=400, height=400))

llm_description = describe_image(copywriter_message_list)

llm_description = extract_product_description(llm_description)
display(HTML(f"<div style='word-wrap: break-word; width: 100;'>{llm_description}</div>"))


##### Reviewer

In [ ]:
with open('prompts/reviewer_system_prompt_v2.txt', 'r') as file:
    reviewer_system_prompt_v2 = file.read()

with open('prompts/reviewer_user_prompt_a_v2.txt', 'r') as file:
    reviewer_user_prompt_a_v2 = file.read()

with open('prompts/reviewer_user_prompt_b_v2.txt', 'r') as file:
    reviewer_user_prompt_b_v2 = file.read()

In [8]:
def review_copy(row, llm_description):
    messages = [
        {
            "role": "system",
            "content": reviewer_system_prompt_v2
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": reviewer_user_prompt_a_v2.replace("{{PRODUCT_TITLE}}", row['product_title']).replace("{{GENERATED_DESCRIPTION}}", llm_description)}
            ]
        }
    ]
       
    for image_url in row['product_images']:
        messages[1]["content"].append({
            "type": "image_url",
            "image_url": {
                "url": image_url,
            }
        })

    messages[1]["content"].append({"type": "text", "text": reviewer_user_prompt_b_v2})

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            max_tokens=4000,
            temperature=0.8 
        )
        return response.choices[0].message.content
    except Exception as e:
        return str(e)

In [ ]:
review_copy(llm_description)

In [ ]:
## cost (12 examples >> 60 images)

## time: 30~60s
## input_tokens = 55567 >> $0.2778
## output_tokens = 104 >> 0.00156

#### Claude

In [ ]:
df_example = pd.read_csv('claude/examples_base64.csv')
df = pd.read_csv('claude/test_products_base64.csv')

def create_example_prompt(df):
    example_prompt = []

    for index, row in df.iterrows():
        product_title = row['product_title']
        product_images_base64 = eval(row['product_images_base64'])
        media_type = eval(row['media_type'])
        product_description = row['description']

        user_content = [
            {"type": "text", "text": "Product Title:"},
            {"type": "text", "text": product_title},
            {"type": "text", "text": "Product Images:"}
        ]

        for img_base64, m_type in zip(product_images_base64, media_type):
            user_content.append({"type": "image", "source": {"type": "base64", "media_type": m_type, "data": img_base64}})

        user_content.append({"type": "text", "text": "Based on the product images, product title, and guidelines provided, write a compelling product description. Present your final product description within <tagline> and <why_youll_love_it> tags."})

        example_prompt.append({
            "role": "user",
            "content": user_content
        })

        example_prompt.append({
            "role": "assistant",
            "content": [
                {"type": "text", "text": product_description}
            ]
        })
    
    return example_prompt

def create_user_prompt(row):

    acutal_prompt = []

    product_title = row['product_title']
    product_images_base64 = eval(row['product_images_base64'])
    media_type = eval(row['media_type'])
    
    user_content = [
        {"type": "text", "text": "Product Title:"},
        {"type": "text", "text": product_title},
        {"type": "text", "text": "Product Images:"}
    ]
    
    for img_base64, m_type in zip(product_images_base64, media_type):
        user_content.append({
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": m_type,
                "data": img_base64
            }
        })

    user_content.append({"type": "text", "text": "Based on the product images, product title, and guidelines provided, write a compelling product description. Present your final product description within <tagline> and <why_youll_love_it> tags."})
    
    acutal_prompt.append({
            "role": "user",
            "content": user_content
        })
    
    
    return acutal_prompt

In [ ]:
prompt_example = create_example_prompt(df_example)
first_product = df.iloc[0]
acutal_prompt = create_user_prompt(first_product)
acutal_prompt

message_list = prompt_example + acutal_prompt
message_list

In [ ]:
from anthropic import Anthropic
load_dotenv()
client = Anthropic()
MODEL_NAME = "claude-3-5-sonnet-20240620"

In [ ]:
response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=4096,
    system = system_prompt,
    messages = message_list, 
    temperature=1
)

print(response.content[0].text)


In [ ]:
# Visualize the prompt (message_list)
# https://jsoneditoronline.org/ 

import random
import string

def generate_random_string(length=10):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

for message in message_list:
    if message['role'] == 'user':
        for content in message['content']:
            if content['type'] == 'image':
                content['source']['data'] = generate_random_string()

updated_message_list = message_list
updated_message_list


## Automated & Structural Output

In [9]:
with open('prompts/copywriting_system_prompt_v2.txt', 'r') as file:
    copywriting_system_prompt_v2 = file.read()

with open('prompts/reviewer_system_prompt_v2.txt', 'r') as file:
    reviewer_system_prompt_v2 = file.read()

with open('prompts/reviewer_user_prompt_a_v2.txt', 'r') as file:
    reviewer_user_prompt_a_v2 = file.read()

with open('prompts/reviewer_user_prompt_b_v2.txt', 'r') as file:
    reviewer_user_prompt_b_v2 = file.read()


examples_df = pd.read_csv('examples.csv')
examples_df['product_images'] = examples_df['product_images'].apply(convert_string_to_list)
example_prompt = create_example_prompt(examples_df)

products_df = pd.read_csv('test_products.csv')
products_df['product_images'] = products_df['product_images'].apply(convert_string_to_list)
products_df.head()

,address,product_title,product_images
0,https://www.pazzion.com/products/c2257-3-joyce...,Joyce Pearl Patent Slingback Heels,[https://www.pazzion.com/cdn/shop/files/JoyceP...
1,https://www.pazzion.com/products/122-5-maia-pe...,Maia Pearl Décor Bow Slingbacks Kitten Heels,[https://www.pazzion.com/cdn/shop/files/MaiaPe...
2,https://www.pazzion.com/products/6313-1-aureli...,Aurelia Strappy Espadrilles,[https://www.pazzion.com/cdn/shop/files/Aureli...
3,https://www.pazzion.com/products/1919-1-margot...,Margot Tweed Lace-up Espadrilles,[https://www.pazzion.com/cdn/shop/files/Margot...
4,https://www.pazzion.com/products/3899-9-fallon...,Fallon Leather Ankle Boots,[https://www.pazzion.com/cdn/shop/files/Fallon...


In [11]:
for index, row in products_df.iterrows():

    print(f"Product Title: {row['product_title']}")
    print(f"Address: {row['address']}")
    print("Product Images:")
    for image_url in row['product_images']:
        print(image_url)
    
    display(Image(url=row['product_images'][0], width=400, height=400))

    user_prompt = create_user_prompt(row)
    message_list = [{"role": "system", "content": copywriting_system_prompt_v2}] + example_prompt + user_prompt

    copywriter_output = describe_image(message_list)
    copywriter_output = extract_product_description(copywriter_output)
    
    print("copywriter_output:")
    display(HTML(f"<div style='word-wrap: break-word; width: 600;'>{copywriter_output}</div>"))
    print("-" * 40)
    products_df.at[index, 'copywriter_output'] = copywriter_output

    reviewer_output = review_copy(row, copywriter_output)
    reviewer_output = extract_product_description(reviewer_output)

    print("reviewer_output:")
    display(HTML(f"<div style='word-wrap: break-word; width: 600;'>{reviewer_output}</div>"))
    print("-" * 40)
    products_df.at[index, 'reviewer_output'] = reviewer_output


Product Title: Joyce Pearl Patent Slingback Heels
Address: https://www.pazzion.com/products/c2257-3-joyce-pearl-patent-slingback-heels
Product Images:
https://www.pazzion.com/cdn/shop/files/JoycePearlPatentSlingbackHeels-Beige-1.jpg?v=1705898929
https://www.pazzion.com/cdn/shop/files/JoycePearlPatentSlingbackHeels-Beige-2.jpg?v=1705898934
https://www.pazzion.com/cdn/shop/files/JoycePearlPatentSlingbackHeels-Beige-3.jpg?v=1705898939
https://www.pazzion.com/cdn/shop/files/JoycePearlPatentSlingbackHeels-Beige-4.jpg?v=1705898944
https://www.pazzion.com/cdn/shop/files/JoycePearlPatentSlingbackHeels-Beige-5.jpg?v=1705898949


copywriter_output:


----------------------------------------
reviewer_output:


----------------------------------------
Product Title: Maia Pearl Décor Bow Slingbacks Kitten Heels
Address: https://www.pazzion.com/products/122-5-maia-pearl-decor-bow-slingbacks-kitten-heels
Product Images:
https://www.pazzion.com/cdn/shop/files/MaiaPearlDi_corBowSlingbacksKittenHeels-Beige-1.webp?v=1705903988
https://www.pazzion.com/cdn/shop/files/MaiaPearlDi_corBowSlingbacksKittenHeels-Beige-2.webp?v=1705903993
https://www.pazzion.com/cdn/shop/files/MaiaPearlDi_corBowSlingbacksKittenHeels-Beige-3.webp?v=1705903999
https://www.pazzion.com/cdn/shop/files/MaiaPearlDi_corBowSlingbacksKittenHeels-Beige-4.webp?v=1705904009
https://www.pazzion.com/cdn/shop/files/MaiaPearlDi_corBowSlingbacksKittenHeels-Beige-5.webp?v=1705904014


copywriter_output:


...

----------------------------------------
reviewer_output:


----------------------------------------
Product Title: Ayla Round-Toe Mary Janes
Address: https://www.pazzion.com/products/3020-1-ayla-round-toe-mary-janes
Product Images:
https://www.pazzion.com/cdn/shop/files/AylaRound-ToeMaryJanes-Beige-1.webp?v=1705649332
https://www.pazzion.com/cdn/shop/files/AylaRound-ToeMaryJanes-Beige-6.webp?v=1715764127
https://www.pazzion.com/cdn/shop/files/AylaRound-ToeMaryJanes-Beige-3.webp?v=1715764127
https://www.pazzion.com/cdn/shop/files/AylaRound-ToeMaryJanes-Beige-4.webp?v=1715764127
https://www.pazzion.com/cdn/shop/files/AylaRound-ToeMaryJanes-Beige-5.webp?v=1715764127


copywriter_output:


----------------------------------------
reviewer_output:


----------------------------------------


In [12]:
products_df.to_csv('products_llm_output_v2.csv', index=False, encoding='utf-8')

In [ ]:
products_df.head()